In [18]:
import numpy as np
import pandas as pd 
from sklearn import preprocessing

In [34]:
#add some feature engineering 
from scipy import sparse as ssp
from scipy.stats import spearmanr
from sklearn.decomposition import PCA, FastICA,TruncatedSVD,NMF
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold,StratifiedKFold

In [6]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

#apply lableEncoder 
for line in train_df.columns:
    if train_df[line].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[line].values) + list(test_df[line].values))
        train_df[line] = lbl.transform(train_df[line].values)
        test_df[line] = lbl.transform(list(test_df[line].values))
        
print('Shape train: {}\nShape test: {}'.format(train_df.shape, test_df.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [24]:
from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train_df.drop(["y"], axis=1))
pca2_results_test = pca.transform(test_df)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train_df.drop(["y"], axis=1))
ica2_results_test = ica.transform(test_df)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train_df['pca_' + str(i)] = pca2_results_train[:,i-1]
    test_df['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    train_df['ica_' + str(i)] = ica2_results_train[:,i-1]
    test_df['ica_' + str(i)] = ica2_results_test[:, i-1]
    
smooth=5
y_train = train_df["y"]
y_train = np.log(y_train+smooth)
y_mean = np.mean(y_train)
y_train = y_train - y_mean 
train_df["y"] = y_train 
#y_mean
#train_df

In [36]:
#train_df.columns.values 

In [28]:
X = train_df.drop('y', axis=1).copy(deep=True)
X_t = test_df.copy(deep=True)

In [38]:
scaler = preprocessing.MinMaxScaler()
X['ID'] = scaler.fit_transform(X[['ID']].values)
X_t['ID'] = scaler.transform(X_t[['ID']].values)
print('feature 1')
sum_cols = []
for c in X.columns:
    score = (spearmanr(y_train,X[c]))
    if score[0]>=0.2 and score[0]<=0.3:
        print(c,score)
        sum_cols.append(c)

X['sum_row_2_to_3'] = X.drop('ID', axis=1)[sum_cols].sum(axis=1)
X_t['sum_row_2_to_3'] = X_t.drop('ID', axis=1)[sum_cols].sum(axis=1)
print('new feature',spearmanr(y_train,X['sum_row_2_to_3']))

feature 1
X14 SpearmanrResult(correlation=0.23271157178351343, pvalue=7.2268888958531111e-53)
X48 SpearmanrResult(correlation=0.20357135784009164, pvalue=1.2996370177538068e-40)
X51 SpearmanrResult(correlation=0.2611090689430815, pvalue=1.4576088840370931e-66)
X66 SpearmanrResult(correlation=0.21315967647205705, pvalue=1.8967274267969423e-44)
X113 SpearmanrResult(correlation=0.20357135784009164, pvalue=1.2996370177538068e-40)
X118 SpearmanrResult(correlation=0.27089687911170574, pvalue=1.0676923363400654e-71)
X119 SpearmanrResult(correlation=0.27089687911170574, pvalue=1.0676923363400654e-71)
X126 SpearmanrResult(correlation=0.24172003199564598, pvalue=5.0736755619928469e-57)


/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2569: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2570: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


X130 SpearmanrResult(correlation=0.23415408749018998, pvalue=1.6054655030408509e-53)
X134 SpearmanrResult(correlation=0.20357135784009164, pvalue=1.2996370177538068e-40)
X147 SpearmanrResult(correlation=0.20357135784009164, pvalue=1.2996370177538068e-40)
X179 SpearmanrResult(correlation=0.23666393517075276, pvalue=1.1433060738837628e-54)
X191 SpearmanrResult(correlation=0.22965447867367564, pvalue=1.693718142409307e-51)
X198 SpearmanrResult(correlation=0.20553581089858822, pvalue=2.2052103895266728e-41)
X222 SpearmanrResult(correlation=0.20357135784009164, pvalue=1.2996370177538068e-40)
X223 SpearmanrResult(correlation=0.22158289577218154, pvalue=5.62402980674849e-48)
X224 SpearmanrResult(correlation=0.22029774009300782, pvalue=1.9859079274389972e-47)
X251 SpearmanrResult(correlation=0.23129920010352548, pvalue=3.1212180651895575e-52)
X264 SpearmanrResult(correlation=0.24384865853019391, pvalue=4.9908281952374735e-58)
X275 SpearmanrResult(correlation=0.27138628318725722, pvalue=5.83417

In [40]:
print('feature 2')
sum_cols = []
for c in X.columns:
    score = (spearmanr(y_train,X[c]))
    if score[0]>=0.1 and score[0]<=0.2:
        print(c,score)
        sum_cols.append(c)

X['sum_row_1_to_2'] = X.drop('ID', axis=1)[sum_cols].sum(axis=1)
X_t['sum_row_1_to_2'] = X_t.drop('ID', axis=1)[sum_cols].sum(axis=1)
print('new feature',spearmanr(y_train,X['sum_row_1_to_2']))

feature 2
X2 SpearmanrResult(correlation=0.13158960738950537, pvalue=1.0181363130966923e-17)
X47 SpearmanrResult(correlation=0.1212883321105124, pvalue=2.8886066381250153e-15)
X52 SpearmanrResult(correlation=0.19736322675451487, pvalue=3.13469444203809e-38)
X64 SpearmanrResult(correlation=0.10181248528247661, pvalue=3.5794673112564369e-11)
X68 SpearmanrResult(correlation=0.16200292022368409, pvalue=3.8076199582459172e-26)
X71 SpearmanrResult(correlation=0.14890315297151863, pvalue=2.6887569017290762e-22)
X75 SpearmanrResult(correlation=0.15457622325411347, pvalue=6.3642766285106105e-24)
X84 SpearmanrResult(correlation=0.14890315297151863, pvalue=2.6887569017290762e-22)
X85 SpearmanrResult(correlation=0.1281335714190161, pvalue=7.1306865445296541e-17)
X96 SpearmanrResult(correlation=0.15027972353435443, pvalue=1.0984648193439938e-22)


/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2569: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2570: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


X150 SpearmanrResult(correlation=0.15828600410347105, pvalue=5.090268480284762e-25)
X151 SpearmanrResult(correlation=0.10642054429461194, pvalue=4.4582130189072218e-12)
X155 SpearmanrResult(correlation=0.13241347559222771, pvalue=6.3524766888659539e-18)
X156 SpearmanrResult(correlation=0.15733046650908872, pvalue=9.8144690505536285e-25)
X170 SpearmanrResult(correlation=0.18496198248759121, pvalue=1.0468008073693234e-33)
X180 SpearmanrResult(correlation=0.13961248645681151, pvalue=9.0733009614535791e-20)
X187 SpearmanrResult(correlation=0.17533512141422311, pvalue=2.078494726313512e-30)
X197 SpearmanrResult(correlation=0.10093443054165158, pvalue=5.2691759331188333e-11)
X208 SpearmanrResult(correlation=0.10268676397658415, pvalue=2.4277321357680762e-11)
X221 SpearmanrResult(correlation=0.10490322566170926, pvalue=8.9409910305564303e-12)
X228 SpearmanrResult(correlation=0.13851144976629962, pvalue=1.7635537001392328e-19)
X241 SpearmanrResult(correlation=0.12792882044810919, pvalue=7.9891

In [41]:
print('feature 3')
sum_cols = []
for c in X.columns:
    score = (spearmanr(y_train,X[c]))
    if score[0]>=0.05 and score[0]<=0.1:
        print(c,score)
        sum_cols.append(c)

X['sum_row_05_to_1'] = X.drop('ID', axis=1)[sum_cols].sum(axis=1)
X_t['sum_row_05_to_1'] = X_t.drop('ID', axis=1)[sum_cols].sum(axis=1)
print('new feature',spearmanr(y_train,X['sum_row_05_to_1']))

feature 3
X12 SpearmanrResult(correlation=0.088043156674917342, pvalue=1.0566533400454334e-08)
X13 SpearmanrResult(correlation=0.051711095904168933, pvalue=0.00079050075352438587)
X34 SpearmanrResult(correlation=0.064353895964323615, pvalue=2.938686779663853e-05)
X44 SpearmanrResult(correlation=0.090772756263977808, pvalue=3.6465030551708312e-09)
X53 SpearmanrResult(correlation=0.070796895215738662, pvalue=4.2762829146537143e-06)
X55 SpearmanrResult(correlation=0.054065429230942832, pvalue=0.00044959684982717063)
X62 SpearmanrResult(correlation=0.093625344331379354, pvalue=1.1599790476874411e-09)
X69 SpearmanrResult(correlation=0.089711138185718048, pvalue=5.5360387795531804e-09)
X78 SpearmanrResult(correlation=0.061828748835767847, pvalue=5.9723462135892377e-05)
X82 SpearmanrResult(correlation=0.053632950041820129, pvalue=0.00049953348735447701)
X102 SpearmanrResult(correlation=0.070796895215738662, pvalue=4.2762829146537143e-06)
X109 SpearmanrResult(correlation=0.074562368809068205, 

/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2569: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:2570: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


X131 SpearmanrResult(correlation=0.073087231859785565, pvalue=2.068577052818342e-06)
X142 SpearmanrResult(correlation=0.086590434631659927, pvalue=1.8378285352101143e-08)
X163 SpearmanrResult(correlation=0.065827247071032075, pvalue=1.9197195118377821e-05)
X171 SpearmanrResult(correlation=0.089288700822733352, pvalue=6.5279726694704066e-09)
X172 SpearmanrResult(correlation=0.093625344331379354, pvalue=1.1599790476874411e-09)
X176 SpearmanrResult(correlation=0.055956031666974389, pvalue=0.00028121562717078831)
X177 SpearmanrResult(correlation=0.053712224262457441, pvalue=0.0004900094723219743)
X183 SpearmanrResult(correlation=0.070038561030065241, pvalue=5.4128758626020661e-06)
X189 SpearmanrResult(correlation=0.082184393610834336, pvalue=9.3300425624300407e-08)
X199 SpearmanrResult(correlation=0.057012532319473334, pvalue=0.00021500340527230225)
X211 SpearmanrResult(correlation=0.068138965875717258, pvalue=9.6680167166954933e-06)
X214 SpearmanrResult(correlation=0.070796895215738662, p

In [43]:
X.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,ica_7,pca_8,ica_8,pca_9,ica_9,pca_10,ica_10,sum_row_2_to_3,sum_row_1_to_2,sum_row_05_to_1
0,0.000000,37,23,20,0,3,27,9,14,0,...,0.014502,10.863589,-0.004618,4.346131,0.014550,-1.319238,0.056137,5.009898,37.889326,8.893976
1,0.000713,37,21,22,4,3,31,11,14,0,...,0.006888,-1.262431,-0.006094,2.462597,0.025231,-1.658361,0.063920,4.859025,27.729671,9.702113
2,0.000832,24,24,38,2,3,30,9,23,0,...,0.026389,3.100797,-0.023800,4.501178,0.015271,-1.155297,0.061265,47.586858,53.118676,39.505611
3,0.001069,24,21,38,5,3,30,11,4,0,...,0.027256,-4.996100,0.016496,5.871117,0.025517,0.561077,0.062468,45.468782,39.996656,40.128244
4,0.001544,24,23,38,5,3,14,3,13,0,...,0.032196,-4.607984,-0.001881,5.753915,-0.018426,3.392346,0.031215,47.568540,36.383393,38.275679


In [9]:
#from lightgbm import LGBMRegressor

In [54]:
# prepare dict of params for xgboost to run with
import xgboost as xgb
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(X.drop(['sum_row_2_to_3', 'sum_row_1_to_2','sum_row_05_to_1'], axis=1), y_train)


# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=500, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

[0]	train-rmse:4.63166	test-rmse:4.63166
[50]	train-rmse:3.60594	test-rmse:3.60593
[100]	train-rmse:2.80764	test-rmse:2.80763
[150]	train-rmse:2.18635	test-rmse:2.18633
[200]	train-rmse:1.70288	test-rmse:1.70286
[250]	train-rmse:1.32665	test-rmse:1.32662
[300]	train-rmse:1.03396	test-rmse:1.0339
[350]	train-rmse:0.806335	test-rmse:0.806322
[400]	train-rmse:0.629455	test-rmse:0.629497
[450]	train-rmse:0.492133	test-rmse:0.492269
[499]	train-rmse:0.387584	test-rmse:0.387882
500


In [55]:
# check f2-score (to get higher score - increase num_boost_round in previous cell)
from sklearn.metrics import r2_score

# now fixed, correct calculation
print(r2_score(dtrain.get_label(), model.predict(dtrain)))

-10.1518148246


In [50]:
# make predictions and save results
dtest = xgb.DMatrix(X_t)
y_pred = model.predict(dtest)
output = pd.DataFrame({'id': test_df['ID'].astype(np.int32), 'y': y_pred})
output.to_csv('xgboost-newfeature-pca-ica.csv', index=False)